In [ ]:
!pip install -r /content/DINO/requirements.txt

In [ ]:
!pip install yapf==0.40.1

In [ ]:
!pip install 'numpy<1.24'

In [ ]:
# Step 1: Change to the correct directory
%cd /content/DINO/models/dino/ops

# Step 2: Run the setup.py command
!python setup.py build install

In [ ]:
!python test.py

In [ ]:
import sys
sys.path.append('/content/DINO')


In [ ]:
%cd /content/DINO

Testing the Fine-Tuned model

In [ ]:
import os, sys
import torch, json
import numpy as np

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops

In [ ]:
from PIL import Image
import datasets.transforms as T

In [ ]:
%cd /content/DINO/models/dino/ops

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
image = Image.open("/content/DINO/16268.jpg").convert("RGB")

In [ ]:
transform = T.Compose([
    T.RandomResize([800], max_size=1333),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
image, _ = transform(image, None)

In [ ]:
model_config_path = "config/DINO/DINO_4scale.py"
model_checkpoint_path = "/content/DINO/logs/DINO/R50-MS4/checkpoint.pth"


In [ ]:
args = SLConfig.fromfile(model_config_path)
args.device = 'cuda'
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

In [ ]:
output = model.cuda()(image[None].cuda())
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

In [ ]:
thershold = 0.3 # set a thershold

vslzr = COCOVisualizer()

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

box_label = [id2name[int(item)] for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': torch.Tensor([image.shape[1], image.shape[2]]),
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None, dpi=100)